# Feature selection

### What is feature selection?

Feature selection is a method of selecting features from your feature set to be used for modeling. It draws from a set of existing features, so it's different than feature engineering because it doesn't create new features. The goal of feature selection is to improve your model's performance. Perhaps your existing feature set is much too large, or some of the features you're working with are unnecessary.

### Removing features manually

One of the easiest ways to determine if a feature is unnecessary is if it is redundant in some way. For example, if it exists in another form as another feature—sometimes, when you create features through feature engineering, you end up duplicating existing features in some way. 

For example, if your dataset contains repeated information in its feature set, it's unlikely you'll need to use each feature for modeling. You may see columns related to city, state, latitude, and longitude in the same dataset, such as in the `volunteer` dataset:

In [1]:
import pandas as pd

dir_string = "../../datasets/"

volunteer = pd.read_csv(dir_string + "volunteer.csv")
volunteer.columns

Index(['opportunity_id', 'content_id', 'vol_requests', 'event_time', 'title',
       'hits', 'summary', 'is_priority', 'category_id', 'category_desc',
       'amsl', 'amsl_unit', 'org_title', 'org_content_id', 'addresses_count',
       'locality', 'region', 'postalcode', 'primary_loc', 'display_url',
       'recurrence_type', 'hours', 'created_date', 'last_modified_date',
       'start_date_date', 'end_date_date', 'status', 'Latitude', 'Longitude',
       'Community Board', 'Community Council ', 'Census Tract', 'BIN', 'BBL',
       'NTA'],
      dtype='object')

The `volunteer` dataset contains a lot of different information related to the location of the volunteer opportunity: `Latitude`, `Longitude`, `locality`, `region`, and `postalcode`.

Dropping columns is as simple as using pandas' `drop()` method, which we learned about previously, but it's important to remember here. Specifying `axis=1` ensures that we drop entire columns:

In [2]:
volunteer_subset = volunteer.drop(["Latitude", "Longitude", "locality", "postalcode"], axis=1)

print(volunteer_subset.columns)

Index(['opportunity_id', 'content_id', 'vol_requests', 'event_time', 'title',
       'hits', 'summary', 'is_priority', 'category_id', 'category_desc',
       'amsl', 'amsl_unit', 'org_title', 'org_content_id', 'addresses_count',
       'region', 'primary_loc', 'display_url', 'recurrence_type', 'hours',
       'created_date', 'last_modified_date', 'start_date_date',
       'end_date_date', 'status', 'Community Board', 'Community Council ',
       'Census Tract', 'BIN', 'BBL', 'NTA'],
      dtype='object')


Perhaps, for your modeling task, you only need the high-level state information in the `region` field, which we didn't drop. 

Another situation where duplicate features occur is through feature engineering. In an earlier example, we took an average on running times from a group of people to use as an aggregate statistic. It's likely that we could drop the values that generated the aggregate statistic. It's very dependent on the model you're trying to build.

### Removing correlated features

Another clear situation in which you'd want to drop features is when they are statistically correlated, meaning they move together directionally. Linear models in particular assume that features are independent of each other, and if features are strongly correlated, that could introduce bias into your model. 

We'll use Pearson's correlation coefficient to check a feature set for correlation. The Pearson correlation coefficient is a measure of this directionality: a score closer to 1 between pairs of features means that they move together in the same direction more strongly, a score closer to 0 means features are not correlated, and a score close to -1 means they are more strongly negatively correlated, meaning one feature increases in value while the other decreases. 

Let's take a look at the `wine` dataset, which is made up of continuous, numerical features of various characteristics of wine:

In [3]:
wine = pd.read_csv(dir_string + "wine_types.csv")

wine.head()

,Type,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


We can easily check correlation in pandas using the `corr()` method, which outputs features and their measures of correlation. Let's use a subset of the `wine` dataset so we can see the output more easily:

In [4]:
wine_subset = wine[["Flavanoids", "Total phenols", "Malic acid", "OD280/OD315 of diluted wines", "Hue"]]

wine_subset.corr()

,Flavanoids,Total phenols,Malic acid,OD280/OD315 of diluted wines,Hue
Flavanoids,1.000000,0.864564,-0.411007,0.787194,0.543479
Total phenols,0.864564,1.000000,-0.335167,0.699949,0.433681
Malic acid,-0.411007,-0.335167,1.000000,-0.368710,-0.561296
OD280/OD315 of diluted wines,0.787194,0.699949,-0.368710,1.000000,0.565468
Hue,0.543479,0.433681,-0.561296,0.565468,1.000000


Taking a close look at the `Flavanoids` feature after correlation, you can see that it's strongly positively correlated with both `Total phenols` and `OD280/OD315 of diluted wines`. Let's drop it from that subset, and then look at the correlation again:

In [5]:
wine_subset = wine_subset.drop("Flavanoids", axis=1)

wine_subset.corr()

,Total phenols,Malic acid,OD280/OD315 of diluted wines,Hue
Total phenols,1.000000,-0.335167,0.699949,0.433681
Malic acid,-0.335167,1.000000,-0.368710,-0.561296
OD280/OD315 of diluted wines,0.699949,-0.368710,1.000000,0.565468
Hue,0.433681,-0.561296,0.565468,1.000000


Another feature you might consider dropping is `OD280/OD315 of diluted wines`, which is strongly positively correlated with `Total Phenols` and moderately positively correlated with `Hue`. This is a situation where you'd want to iterate on your chosen features and see how they impact your model's performance.

### Dimensionality reduction for feature selection

A less manual way of reducing the size of your feature set is through dimensionality reduction. Dimensionality reduction is a form of unsupervised learning that transforms your data in a way that shrinks the number of features in your feature space. 

The method of dimensionality reduction we'll cover is principal component analysis, or PCA. PCA uses a linear transformation to project features into a space where they are completely uncorrelated. While the feature space is reduced, the variance is captured in a meaningful way by combining features into components. PCA captures, in each component, as much of the variance in the dataset as possible. In terms of feature selection, it can be a useful method when you have a large number of features and no strong candidates for elimination.

Transforming a dataset through PCA is relatively straightforward in scikit-learn. Let's apply PCA to the `wine` dataset, to see if we can get an increase in our model's accuracy. 

First, we'll set up our data for modeling by removing the label column, `Type`, from the `wine` dataset:

In [6]:
wine_X = wine.drop("Type", axis=1)

Next, we'll apply `PCA` to `wine_X` using `fit_transform()`, which transforms the data into components:

In [7]:
from sklearn.decomposition import PCA

pca = PCA()
transformed_X = pca.fit_transform(wine_X)

By default, PCA in scikit-learn keeps the number of components equal to the number of input features. If we print out the explained variance ratio, we can see, by component, the percentage of variance explained by that component:

In [8]:
pca.explained_variance_ratio_

array([9.98091230e-01, 1.73591562e-03, 9.49589576e-05, 5.02173562e-05,
       1.23636847e-05, 8.46213034e-06, 2.80681456e-06, 1.52308053e-06,
       1.12783044e-06, 7.21415811e-07, 3.78060267e-07, 2.12013755e-07,
       8.25392788e-08])

You can see that much of the variance is explained by the first component here—around 99%—so it's likely that we could drop those components that don't explain much variance.

There are a couple of things to note regarding PCA. The first is that it can be very difficult to interpret PCA components beyond which components explain the most variance. PCA is more of a black box method than other methods of dimensionality reduction. The other thing to note is that PCA is a good step to do at the end of your preprocessing journey, because of the way the data gets transformed and reshaped. It can be difficult to do much feature work post-PCA, other than eliminating components that aren't useful in explaining variance.

### Training a model using PCA

Let's look at training a model both with the components from PCA, as well as without, to compare model performance.

Let's first see how modeling without PCA performs. We'll use a basic k-nearest neighboors model to see if we can predict the wine `Type` from its other features. First, let's set up the data. We'll split the non-transformed data into training and test sets:

In [9]:
from sklearn.model_selection import train_test_split

y = wine["Type"]
X_wine_train, X_wine_test, y_wine_train, y_wine_test = train_test_split(wine_X, y)

Next, we'll create the `knn` classifier, fit it to the training data, and score it on the test data:

In [10]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(X_wine_train, y_wine_train)

print(knn.score(X_wine_test, y_wine_test))

0.6666666666666666


So it doesn't perform that badly without PCA. 

Let's take a look at how `knn` performs using PCA components as the `X` data. We can simply pass in the `transformed_X` data we created in the previous step to `train_test_split()` to create training and test data. We'll also create a new `knn` model to use on the component data, and then print out the score:

In [11]:
X_pca_train, X_pca_test, y_pca_train, y_pca_test = train_test_split(transformed_X, y)

knn_pca = KNeighborsClassifier()
knn_pca.fit(X_pca_train, y_pca_train)

print(knn_pca.score(X_pca_test, y_pca_test))

0.6


There's likely an improvement, without doing any other feature selection. We could definitely remove correlated features, or even scale the data, before transforming the data via PCA to get additional modeling improvements.